In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [11]:
cd /content/drive/My\ Drive/Matrix\ Transformacja/dw_matrix

/content/drive/My Drive/Matrix Transformacja/dw_matrix


In [14]:
cd matrix_one

/content/drive/My Drive/Matrix Transformacja/dw_matrix/matrix_one


In [0]:
df = pd.read_csv('../data/shoes_prices.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [24]:
df['brand_cat'] = df['brand'].factorize()[0]

run_model(['brand_cat'])

(-58.38655694633361, 4.223555478221712)

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'])

In [31]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]

run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [28]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [33]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [39]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'
literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [0]:
def parse_features(x):
  output_dict = {}

  if str(x) == 'nan':
    return output_dict
  
  features = literal_eval(x.replace('\\"', '"'))

  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [50]:
df['features_parsed'].head()

0    {'gender': 'men', 'shoe size': 'm', 'shoe cate...
1    {'gender': 'men', 'shoe size': 'm', 'shoe cate...
2    {'gender': 'men', 'color': 'black', 'shipping ...
3    {'gender': 'men', 'color': 'black', 'shipping ...
4    {'gender': 'men', 'color': 'black', 'shipping ...
Name: features_parsed, dtype: object

In [51]:
keys = set()


df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [52]:
df.features_parsed.head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [56]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan )


In [57]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_tactical', 'feat_insulated', 'feat_dimensions',
       'feat_material type', 'feat_product #', 'feat_shipping to usa',
       'feat_resolution megapixels', 'feat_lens color family',
       'feat_characteristics', 'feat_safety features'],
      dtype='object', length=526)

In [59]:
df[ False == df['feat_athlete'].isnull() ].shape[0] # SŁABO

8

In [156]:
df['feat_resolution megapixels'].value_counts()

2 mp    2
Name: feat_resolution megapixels, dtype: int64

In [0]:
keys_stat = {}

for key in keys:
  keys_stat[key] = df[ False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [67]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [157]:
df[get_name_feat('brand') + '_cat'] = df[get_name_feat('brand')].map(lambda x: str(x).lower()).factorize()[0]
df[get_name_feat('color') + '_cat'] = df[get_name_feat('color')].map(lambda x: str(x).lower()).factorize()[0]
df[get_name_feat('gender') + '_cat'] = df[get_name_feat('gender')].map(lambda x: str(x).lower()).factorize()[0]
df[get_name_feat('manufacturer part number') + '_cat'] = df[get_name_feat('manufacturer part number')].map(lambda x: str(x).lower()).factorize()[0]
df[get_name_feat('material') + '_cat'] = df[get_name_feat('material')].map(lambda x: str(x).lower()).factorize()[0]

df[get_name_feat('shape') + '_cat'] = df[get_name_feat('shape')].map(lambda x: str(x).lower()).factorize()[0]
df[get_name_feat('sport') + '_cat'] = df[get_name_feat('sport')].map(lambda x: str(x).lower()).factorize()[0]
df[get_name_feat('style') + '_cat'] = df[get_name_feat('style')].map(lambda x: str(x).lower()).factorize()[0]
df[get_name_feat('condition') + '_cat'] = df[get_name_feat('condition')].map(lambda x: str(x).lower()).factorize()[0]
df[get_name_feat('occasion') + '_cat'] = df[get_name_feat('occasion')].map(lambda x: str(x).lower()).factorize()[0]
df[get_name_feat('age group') + '_cat'] = df[get_name_feat('age group')].map(lambda x: str(x).lower()).factorize()[0]


df[get_name_feat('resolution megapixels') + '_cat'] = df[get_name_feat('resolution megapixels')].map(lambda x: str(x).lower()).factorize()[0]


run_model([get_name_feat('brand') + '_cat'])

(-62.109033101587315, 3.4698205059389093)



> Indented block




In [0]:
df[get_name_feat('age group') + '_cat'] = df[get_name_feat('age group')].map(lambda x: 'men' if x == 'men\'s' else x)
df[get_name_feat('age group') + '_cat'] = df[get_name_feat('age group') + '_cat'].map(lambda x: 'men' if x == 'mens' else x)
df[get_name_feat('age group') + '_cat'] = df[get_name_feat('age group') + '_cat'].map(lambda x: 'adult' if x == 'men||women' else x)
df[get_name_feat('age group') + '_cat'] = df[get_name_feat('age group') + '_cat'].map(lambda x: 'adult' if x == 'women ,�� unisex' else x)
df[get_name_feat('age group') + '_cat'] = df[get_name_feat('age group') + '_cat'].map(lambda x: 'adult' if x == 'adult ,�� teen' else x)
#df[get_name_feat('age group') + '_cat'] = df[get_name_feat('age group')].map(lambda x: 'adult' if x == 'women ,�� unisex' else x)

df[get_name_feat('age group') + '_cat'] = df[get_name_feat('age group') + '_cat'].map(lambda x: str(x).lower()).factorize()[0]

In [123]:
df[get_name_feat('age group') + '_cat'].value_counts()

0     13226
1      4568
2       387
3        77
4         6
6         4
5         4
7         3
10        2
8         2
9         1
Name: feat_age group_cat, dtype: int64

In [73]:
run_model([get_name_feat('brand') + '_cat',
           get_name_feat('color') + '_cat',
           get_name_feat('gender') + '_cat',
           get_name_feat('manufacturer part number') + '_cat',
           get_name_feat('material') + '_cat'])

(-60.71076955603803, 4.22539811801691)

In [0]:
df['brand'] = df['brand'].map(lambda x:str(x).lower())

In [136]:
df[ df['weight'].isnull() ].shape

(18280, 538)

In [0]:
# 1lbs = 454g
# 1 ounce = 28g
# 

def weight_func(weight):
  if weight in [None, 'nan', np.nan]:
    return 0

  try:
    w, metrics = str(weight).split(' ')
  except:
    print(weight)
    weight = 0
  
  metrics = metrics.lower()
  w = float(w)

  if metrics in ['ounce', 'ounces']:
    return int(w * 28)
  if metrics in ['lbs', 'pounds']:
    return int(w * 454)
  if metrics in ['kg']:
    return int(w * 1000)

  return 0

df['weight_cat'] = df['weight'].map(weight_func)

In [152]:
df['weight_cat'].value_counts()

0       17737
454       121
1362       92
1271       70
2270       48
        ...  
367         1
976         1
2769        1
2497        1
3382        1
Name: weight_cat, Length: 66, dtype: int64

In [0]:
model = RandomForestRegressor(max_depth=10, n_estimators=150)
feats = ['brand_cat',
         'feat_gender_cat',
         'feat_material_cat',
         'feat_shape_cat',
         'feat_age group_cat',
         'weight_cat',
         'feat_resolution megapixels_cat'] # Buty z kamerami 2MP! Bez znaczenia dla ceny

result = run_model( feats, model )

In [160]:
X = df[ feats ].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

result
perm = PermutationImportance(m, random_state=1).fit(X,y )
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2716 ± 0.0147,brand_cat
0.1105 ± 0.0078,feat_material_cat
0.0267 ± 0.0043,feat_gender_cat
0.0139 ± 0.0009,feat_shape_cat
0.0059 ± 0.0017,weight_cat
0.0014 ± 0.0003,feat_age group_cat
0 ± 0.0000,feat_resolution megapixels_cat


In [83]:
df['brand'].value_counts(normalize=True)

nike               0.097210
puma               0.033315
ralph lauren       0.028775
vans               0.021116
new balance        0.020295
                     ...   
acne studios       0.000055
1901               0.000055
unbrand            0.000055
latico leathers    0.000055
cocoons            0.000055
Name: brand, Length: 1732, dtype: float64

In [86]:
df[ df['brand'] == 'nike' ].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [0]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   day4.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	Untitled0.ipynb

no changes added to commit (use "git add" and/or "git commit -a")
